# Inference on Predictive and Causal Effects in High-Dimensional Nonlinear Models

## Impact of 401(k) on  Financial Wealth

As a practical illustration of the DML method, we consider estimation of the effect of 401(k) eligibility and participation
on accumulated assets. 401(k) plans are pension accounts sponsored by employers. The key problem in determining the effect of participation in 401(k) plans on accumulated assets is saver heterogeneity coupled with the fact that the decision to enroll in a 401(k) is non-random. It is generally recognized that some people have a higher preference for saving than others. It also seems likely that those individuals with high unobserved preference for saving would be most likely to choose to participate in tax-advantaged retirement savings plans and would tend to have otherwise high amounts of accumulated assets. The presence of unobserved savings preferences with these properties then implies that conventional estimates that do not account for saver heterogeneity and endogeneity of participation will be biased upward, tending to overstate the savings effects of 401(k) participation.

One can argue that eligibility for enrolling in a 401(k) plan in this data can be taken as exogenous after conditioning on a few observables of which the most important for their argument is income. The basic idea is that, at least around the time 401(k)’s initially became available, people were unlikely to be basing their employment decisions on whether an employer offered a 401(k) but would instead focus on income and other aspects of the job.

### Data

The data set can be downloaded from the github repo


In [ ]:
!pip install wget

In [ ]:
# Import relevant packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict, KFold
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV, LinearRegression, Ridge, Lasso, LogisticRegressionCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
import patsy
import warnings
from sklearn.base import BaseEstimator, clone
import statsmodels.api as sm
import statsmodels.formula.api as smf
from IPython.display import Markdown
import wget
import os
import seaborn as sns
warnings.simplefilter('ignore')
np.random.seed(1234)

In [ ]:
file = "https://raw.githubusercontent.com/CausalAIBook/MetricsMLNotebooks/main/data/401k.csv"
data = pd.read_csv(file)

In [ ]:
data.describe()

In [ ]:
data.head()

In [ ]:
readme = "https://raw.githubusercontent.com/CausalAIBook/MetricsMLNotebooks/main/data/401k.md"
filename = wget.download(readme)
display(Markdown(open(filename, 'r').read()))

The data consist of 9,915 observations at the household level drawn from the 1991 Survey of Income and Program Participation (SIPP). We use net financial assets (*net\_tfa*) as the outcome variable, $Y$,  in our analysis. The net financial assets are computed as the sum of IRA balances, 401(k) balances, checking accounts, saving bonds, other interest-earning accounts, other interest-earning assets, stocks, and mutual funds less non mortgage debts.

Among the $9915$ individuals, $3682$ are eligible to participate in the program. The variable *e401* indicates eligibility and *p401* indicates participation, respectively.

In [ ]:
sns.countplot(data, x='e401')
plt.show()

Eligibility is highly associated with financial wealth:

In [ ]:
sns.displot(data=data, x='net_tfa', kind='kde', col='e401', hue='e401', fill=True)
plt.show()

The unconditional APE of e401 is about $19559$:

In [ ]:
e1 = data[data['e401'] == 1]['net_tfa']
e0 = data[data['e401'] == 0]['net_tfa']
print(f'{np.mean(e1) - np.mean(e0):.0f}')

Among the $3682$ individuals that  are eligible, $2594$ decided to participate in the program. The unconditional APE of p401 is about $27372$:

In [ ]:
e1 = data[data['p401'] == 1]['net_tfa']
e0 = data[data['p401'] == 0]['net_tfa']
print(f'{np.mean(e1) - np.mean(e0):.0f}')

As discussed, these estimates are biased since they do not account for saver heterogeneity and endogeneity of participation.

In [ ]:
y = data['net_tfa'].values
D = data['e401'].values
D2 = data['p401'].values
D3 = data['a401'].values
X = data.drop(['e401', 'p401', 'a401', 'tw', 'tfa', 'net_tfa', 'tfa_he',
               'hval', 'hmort', 'hequity',
               'nifa', 'net_nifa', 'net_n401', 'ira',
               'dum91', 'icat', 'ecat', 'zhat',
               'i1', 'i2', 'i3', 'i4', 'i5', 'i6', 'i7',
               'a1', 'a2', 'a3', 'a4', 'a5'], axis=1)
X.columns

### We define a transformer that constructs the engineered features for controls

In [ ]:
!pip install formulaic

In [ ]:
from sklearn.base import TransformerMixin, BaseEstimator
from formulaic import Formula

class FormulaTransformer(TransformerMixin, BaseEstimator):

    def __init__(self, formula, array=False):
        self.formula = formula
        self.array = array

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        df = Formula(self.formula).get_model_matrix(X)
        if self.array:
            return df.values
        return df

In [ ]:
transformer = FormulaTransformer("0 + poly(age, degree=6, raw=True) + poly(inc, degree=8, raw=True) "
                                 "+ poly(educ, degree=4, raw=True) + poly(fsize, degree=2, raw=True) "
                                 "+ male + marr + twoearn + db + pira + hown")

In [ ]:
transformer.fit_transform(X).describe()

In [ ]:
transformer = FormulaTransformer("0 + poly(age, degree=6, raw=True) + poly(inc, degree=8, raw=True) "
                                 "+ poly(educ, degree=4, raw=True) + poly(fsize, degree=2, raw=True) "
                                 "+ male + marr + twoearn + db + pira + hown", array=True)

## Estimating the ATE of 401(k) Eligibility on Net Financial Assets

We are interested in valid estimators of the average treatment effect of `e401` and `p401` on `net_tfa`. We start using ML approaches to estimate the function $g_0$ and $m_0$ in the following PLR model:

\begin{eqnarray}
 &  Y = D\theta_0 + g_0(X) + \zeta,  &  E[\zeta \mid D,X]= 0,\\
 & D = m_0(X) +  V,   &  E[V \mid X] = 0.
\end{eqnarray}

### Double ML in PLR with Cross-Fitting

We define a simple dml function that is parameterized by arbitrary ML models and returns the treatment effect and other useful quantities of the analysis

In [ ]:
def dml(X, D, y, modely, modeld, *, nfolds, classifier=False):
    '''
    DML for the Partially Linear Model setting with cross-fitting

    Input
    -----
    X: the controls
    D: the treatment
    y: the outcome
    modely: the ML model for predicting the outcome y
    modeld: the ML model for predicting the treatment D
    nfolds: the number of folds in cross-fitting
    classifier: bool, whether the modeld is a classifier or a regressor

    Output
    ------
    point: the point estimate of the treatment effect of D on y
    stderr: the standard error of the treatment effect
    yhat: the cross-fitted predictions for the outcome y
    Dhat: the cross-fitted predictions for the treatment D
    resy: the outcome residuals
    resD: the treatment residuals
    epsilon: the final residual-on-residual OLS regression residual
    '''
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=123) # shuffled k-folds
    yhat = cross_val_predict(modely, X, y, cv=cv, n_jobs=-1) # out-of-fold predictions for y
    # out-of-fold predictions for D
    # use predict or predict_proba dependent on classifier or regressor for D
    if classifier:
        Dhat = cross_val_predict(modeld, X, D, cv=cv, method='predict_proba', n_jobs=-1)[:, 1]
    else:
        Dhat = cross_val_predict(modeld, X, D, cv=cv, n_jobs=-1)
    # calculate outcome and treatment residuals
    resy = y - yhat
    resD = D - Dhat

    # final stage ols based point estimate and standard error
    point = np.mean(resy * resD) / np.mean(resD**2)
    epsilon = resy - point * resD
    var = np.mean(epsilon**2 * resD**2) / np.mean(resD**2)**2
    stderr = np.sqrt(var / X.shape[0])

    return point, stderr, yhat, Dhat, resy, resD, epsilon

In [ ]:
def summary(point, stderr, yhat, Dhat, resy, resD, epsilon, X, D, y, *, name):
    '''
    Convenience summary function that takes the results of the DML function
    and summarizes several estimation quantities and performance metrics.
    '''
    return pd.DataFrame({'estimate': point, # point estimate
                         'stderr': stderr, # standard error
                         'lower': point - 1.96*stderr, # lower end of 95% confidence interval
                         'upper': point + 1.96*stderr, # upper end of 95% confidence interval
                         'rmse y': np.sqrt(np.mean(resy**2)), # RMSE of model that predicts outcome y
                         'rmse D': np.sqrt(np.mean(resD**2)), # RMSE of model that predicts treatment D
                         'accuracy D': np.mean(np.abs(resD) < .5), # binary classification accuracy of model for D
                         }, index=[name])

#### Double Lasso with Cross-Fitting

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=123)
lassoy = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv))
lassod = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv))
result = dml(X, D, y, lassoy, lassod, nfolds=5)

In [ ]:
table = summary(*result, X, D, y, name='double lasso')
table

#### Using a Penalized Logistic Regression for D

Note the default logistic regression uses an $\ell_2$ penalty. You can use the $\ell_1$ penalty as well, but computation will take longer.

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=123)
lassoy = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv))
lgrd = make_pipeline(transformer, StandardScaler(), LogisticRegressionCV(cv=cv))
result = dml(X, D, y, lassoy, lgrd, nfolds=5, classifier=True)

In [ ]:
summary(*result,  X, D, y, name='lasso/logistic')

In [ ]:
table = pd.concat([table , summary(*result,  X, D, y, name='lasso/logistic')])
table

Then, we repeat this procedure for various machine learning methods.

### Random Forests

In [ ]:
rfy = make_pipeline(transformer, RandomForestRegressor(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001))
rfd = make_pipeline(transformer, RandomForestClassifier(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001))
result = dml(X, D, y, rfy, rfd, nfolds=5, classifier=True)

In [ ]:
table = pd.concat([table , summary(*result,  X, D, y, name='random forest')])
table

### Decision Trees

In [ ]:
dtry = make_pipeline(transformer, DecisionTreeRegressor(min_samples_leaf=10, ccp_alpha=.001))
dtrd = make_pipeline(transformer, DecisionTreeClassifier(min_samples_leaf=10, ccp_alpha=.001))
result = dml(X, D, y, dtry, dtrd, nfolds=5, classifier=True)

In [ ]:
table = pd.concat([table , summary(*result,  X, D, y, name='decision tree')])
table

### Boosted Trees

In [ ]:
gbfy = make_pipeline(transformer, GradientBoostingRegressor(max_depth=2, n_iter_no_change=5))
gbfd = make_pipeline(transformer, GradientBoostingClassifier(max_depth=2, n_iter_no_change=5))
result = dml(X, D, y, gbfy, gbfd, nfolds=5, classifier=True)

In [ ]:
table = pd.concat([table , summary(*result,  X, D, y, name='boosted forest')])
table

The best model with lowest RMSE in both equation is the PLR model estimated via lasso.

### AutoML

In [ ]:
!pip install flaml

In [ ]:
from flaml import AutoML
flamly = make_pipeline(transformer, AutoML(time_budget=100, task='regression', early_stop=True,
                                    eval_method='cv', n_splits=3, metric='r2', verbose=3))
flamld = make_pipeline(transformer, AutoML(time_budget=100, task='classification', early_stop=True,
                                           eval_method='cv', n_splits=3, metric='r2', verbose=3))
result = dml(X, D, y, flamly, flamld, nfolds=5, classifier=True)

In [ ]:
table = pd.concat([table , summary(*result,  X, D, y, name='automl')])
table

# Semi-Cross-Fitting

To avoid the computational cost of performing model selection within each fold (assuming that we don't select among an exponential set of hyperparameters/models in the number of samples), it is ok to perform model selection using all the data and then perform cross-fitting with the selected model.

In [ ]:
flamly = make_pipeline(transformer, AutoML(time_budget=100, task='regression', early_stop=True,
                                    eval_method='cv', n_splits=3, metric='r2', verbose=0))
flamld = make_pipeline(transformer, AutoML(time_budget=100, task='classification', early_stop=True,
                                           eval_method='cv', n_splits=3, metric='r2', verbose=0))

In [ ]:
flamly.fit(X, y)
besty = make_pipeline(transformer, clone(flamly[-1].best_model_for_estimator(flamly[-1].best_estimator)))

In [ ]:
flamld.fit(X, D)
bestd = make_pipeline(transformer, clone(flamld[-1].best_model_for_estimator(flamld[-1].best_estimator)))

In [ ]:
result = dml(X, D, y, besty, bestd, nfolds=5, classifier=True)

In [ ]:
table = pd.concat([table , summary(*result,  X, D, y, name='automl (semi-cfit)')])
table

### Semi-Crossfitting with Stacking

In [ ]:
def dml_dirty(X, D, y, modely_list, modeld_list, *,
              stacker=LinearRegression(), nfolds, classifier=False):
    '''
    DML for the Partially Linear Model setting with semi-cross-fitting

    Input
    -----
    X: the controls
    D: the treatment
    y: the outcome
    modely: the ML model for predicting the outcome y
    modeld: the ML model for predicting the treatment D
    stacker: model used to aggregate predictions of each of the base models
    nfolds: the number of folds in cross-fitting
    classifier: bool, whether the modeld is a classifier or a regressor

    Output
    ------
    point: the point estimate of the treatment effect of D on y
    stderr: the standard error of the treatment effect
    yhat: the cross-fitted predictions for the outcome y
    Dhat: the cross-fitted predictions for the treatment D
    resy: the outcome residuals
    resD: the treatment residuals
    epsilon: the final residual-on-residual OLS regression residual
    '''
    # construct out-of-fold predictions for each model
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=123)
    yhats = np.array([cross_val_predict(modely, X, y, cv=cv, n_jobs=-1) for modely in modely_list]).T
    if classifier:
        Dhats = np.array([cross_val_predict(modeld, X, D, cv=cv, method='predict_proba', n_jobs=-1)[:, 1]
                         for modeld in modeld_list]).T
    else:
        Dhats = np.array([cross_val_predict(modeld, X, D, cv=cv, n_jobs=-1) for modeld in modeld_list]).T
    # calculate stacked residuals by finding optimal coefficients
    # and weigthing out-of-sample predictions by these coefficients
    yhat = stacker.fit(yhats, y).predict(yhats)
    Dhat = stacker.fit(Dhats, D).predict(Dhats)
    resy = y - yhat
    resD = D - Dhat
    # go with the stacked residuals
    point = np.mean(resy * resD) / np.mean(resD**2)
    epsilon = resy - point * resD
    var = np.mean(epsilon**2 * resD**2) / np.mean(resD**2)**2
    stderr = np.sqrt(var / X.shape[0])
    return point, stderr, yhat, Dhat, resy, resD, epsilon

In [ ]:
result = dml_dirty(X, D, y, [lassoy, rfy, dtry, gbfy], [lgrd, rfd, dtrd, gbfd],
                   nfolds=5, classifier=True)

In [ ]:
table = pd.concat([table , summary(*result, X, D, y, name='stacked (semi-cfit)')])
table

## Interactive Regression Model (IRM)

Next, we consider estimation of average treatment effects when treatment effects are fully heterogeneous:

 \begin{eqnarray}\label{eq: HetPL1}
 & Y  = g_0(D, X) + U,  &  \quad E[U \mid X, D]= 0,\\
  & D  = m_0(X) + V,  & \quad  E[V\mid X] = 0.
\end{eqnarray}

To reduce the disproportionate impact of extreme propensity score weights in the interactive model
we trim the propensity scores which are below .01 or above .99.

In [ ]:
def dr(X, D, y, modely0, modely1, modeld, *, trimming=0.01, nfolds):
    '''
    DML for the Interactive Regression Model setting (Doubly Robust Learning)
    with cross-fitting

    Input
    -----
    X: the controls
    D: the treatment
    y: the outcome
    modely0: the ML model for predicting the outcome y in the control population
    modely1: the ML model for predicting the outcome y in the treated population
    modeld: the ML model for predicting the treatment D
    trimming: threshold below which to trim propensities
    nfolds: the number of folds in cross-fitting

    Output
    ------
    point: the point estimate of the treatment effect of D on y
    stderr: the standard error of the treatment effect
    yhat: the cross-fitted predictions for the outcome y
    Dhat: the cross-fitted predictions for the outcome D
    resy: the outcome residuals
    resD: the treatment residuals
    drhat: the doubly robust quantity for each sample
    '''
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=123)
    yhat0, yhat1 = np.zeros(y.shape), np.zeros(y.shape)
    # we will fit a model E[Y| D, X] by fitting a separate model for D==0
    # and a separate model for D==1.
    for train, test in cv.split(X, y):
        # train a model on training data that received treatment zero and predict on all data in test set
        yhat0[test] = clone(modely0).fit(X.iloc[train][D[train]==0], y[train][D[train]==0]).predict(X.iloc[test])
        # train a model on training data that received treatment one and predict on all data in test set
        yhat1[test] = clone(modely1).fit(X.iloc[train][D[train]==1], y[train][D[train]==1]).predict(X.iloc[test])
    # prediction for observed treatment
    yhat = yhat0 * (1 - D) + yhat1 * D
    # propensity scores
    Dhat = cross_val_predict(modeld, X, D, cv=cv, method='predict_proba', n_jobs=-1)[:, 1]
    Dhat = np.clip(Dhat, trimming, 1 - trimming)
    # doubly robust quantity for every sample
    drhat = yhat1 - yhat0 + (y - yhat) * (D/Dhat - (1 - D)/(1 - Dhat))
    point = np.mean(drhat)
    var = np.var(drhat)
    stderr = np.sqrt(var / X.shape[0])
    return point, stderr, yhat, Dhat, y - yhat, D - Dhat, drhat

**NB**: There is randomness across the random seed that potentially causes instability in our estimates. In particular, we find that the lasso/logistic specification for the IRM model exhibits great instability across different seeds. To mitigate these differences, we can take an average across seeds. In principle we would do this for all estimates, but for computation and simplicity, we cheat a little bit and do it only for the model specification we know varies significantly.

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=123)
lassoytest = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv))
lgrdtest = make_pipeline(transformer, StandardScaler(), LogisticRegressionCV(cv=cv))
result = dr(X, D, y, lassoytest, lassoytest, lgrdtest, nfolds=5)
seed_estimates = summary(*result,  X, D, y, name='lasso/logistic')

for i in range(9):
  cv = KFold(n_splits=5, shuffle=True, random_state=i)
  lassoytest = make_pipeline(transformer, StandardScaler(), LassoCV(cv=cv))
  lgrdtest = make_pipeline(transformer, StandardScaler(), LogisticRegressionCV(cv=cv))
  result = dr(X, D, y, lassoytest, lassoytest, lgrdtest, nfolds=5)
  seed_estimates = pd.concat([seed_estimates , summary(*result,  X, D, y, name='lasso/logistic')])

seed_estimates

As we can see, using lasso and logistic regression for $Y$ and $D$ respectively leads to estimates that vary across different seeds. Letting $\hat{\theta}_s$ denote the vector of $s=10$ estimates across the seeds, we take $$\hat{\theta} := \text{median}(\hat{\theta}_s)$$ and $$SE(\hat{\theta}) := \sqrt{\text{median}\left(SE(\hat{\theta}_s)^2 + (\hat{\theta}_s - \hat{\theta})^2\right)}$$ (Note in the standard error calculation, the operation of de-medianing $\hat{\theta}_s - \hat{\theta}$ is broadcasted.)

In [ ]:
med_theta = np.median(seed_estimates.values[:,0])
se_med = np.sqrt(np.median((seed_estimates.values[:,1])**2 + (seed_estimates.values[:,0] - med_theta)**2))
tabledr = pd.DataFrame({'estimate': med_theta,
                        'stderr': se_med,
                        'lower': med_theta - 1.96*se_med,
                        'upper': med_theta + 1.96*se_med,
                        'rmse y': np.median(seed_estimates.values[:,4]),
                        'rmse D': np.median(seed_estimates.values[:,5]),
                        'accuracy D': np.median(seed_estimates.values[:,6]),
                        }, index=['lasso/logistic'])

In [ ]:
rfy = make_pipeline(transformer, RandomForestRegressor(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001))
rfd = make_pipeline(transformer, RandomForestClassifier(n_estimators=100, min_samples_leaf=10, ccp_alpha=.001))
result = dr(X, D, y, rfy, rfy, rfd, nfolds=5)

In [ ]:
tabledr = pd.concat([tabledr , summary(*result,  X, D, y, name='random forest')])
tabledr

In [ ]:
dtry = make_pipeline(transformer, DecisionTreeRegressor(min_samples_leaf=10, ccp_alpha=.001))
dtrd = make_pipeline(transformer, DecisionTreeClassifier(min_samples_leaf=10, ccp_alpha=.001))
result = dr(X, D, y, dtry, dtry, dtrd, nfolds=5)

In [ ]:
tabledr = pd.concat([tabledr , summary(*result,  X, D, y, name='decision tree')])
tabledr

In [ ]:
gbfy = make_pipeline(transformer, GradientBoostingRegressor(max_depth=2, n_iter_no_change=5))
gbfd = make_pipeline(transformer, GradientBoostingClassifier(max_depth=2, n_iter_no_change=5))
result = dr(X, D, y, gbfy, gbfy, gbfd, nfolds=5)

In [ ]:
tabledr = pd.concat([tabledr , summary(*result, X, D, y, name='boosted forest')])
tabledr

These estimates that flexibly account for confounding are substantially attenuated relative to the baseline estimate (19559) that does not account for confounding. They suggest much smaller causal effects of 401(k) eligiblity on financial asset holdings.

# Semi-Cross-Fitting

In [ ]:
from flaml import AutoML
flamly0 = make_pipeline(transformer, AutoML(time_budget=60, task='regression', early_stop=True,
                                     eval_method='cv', n_splits=3, metric='r2', verbose=0))
flamly1 = make_pipeline(transformer, AutoML(time_budget=60, task='regression', early_stop=True,
                                     eval_method='cv', n_splits=3, metric='r2', verbose=0))
flamld = make_pipeline(transformer, AutoML(time_budget=60, task='classification', early_stop=True,
                                           eval_method='cv', n_splits=3, metric='r2', verbose=0))

In [ ]:
flamly0.fit(X[D==0], y[D==0])
besty0 = make_pipeline(transformer, clone(flamly0[-1].best_model_for_estimator(flamly0[-1].best_estimator)))

In [ ]:
flamly1.fit(X[D==1], y[D==1])
besty1 = make_pipeline(transformer, clone(flamly1[-1].best_model_for_estimator(flamly1[-1].best_estimator)))

In [ ]:
flamld.fit(X, D)
bestd = make_pipeline(transformer, clone(flamld[-1].best_model_for_estimator(flamld[-1].best_estimator)))

In [ ]:
result = dr(X, D, y, besty0, besty1, bestd, nfolds=5)

In [ ]:
tabledr = pd.concat([tabledr , summary(*result,  X, D, y, name='automl (semi-cfit)')])
tabledr

In [ ]:
def dr_dirty(X, D, y, modely0_list, modely1_list, modeld_list, *,
             stacker=LinearRegression(), trimming=0.01, nfolds):
    '''
    DML for the Interactive Regression Model setting (Doubly Robust Learning)
    with cross-fitting

    Input
    -----
    X: the controls
    D: the treatment
    y: the outcome
    modely_list: list of ML models for predicting the outcome y
    modeld_list: list of ML models for predicting the treatment D
    stacker: model used to aggregate predictions of each of the base models
    trimming: threshold below which to trim propensities
    nfolds: the number of folds in cross-fitting

    Output
    ------
    point: the point estimate of the treatment effect of D on y
    stderr: the standard error of the treatment effect
    yhat: the cross-fitted predictions for the outcome y
    Dhat: the cross-fitted predictions for the outcome D
    resy: the outcome residuals
    resD: the treatment residuals
    drhat: the doubly robust quantity for each sample
    '''
    cv = KFold(n_splits=nfolds, shuffle=True, random_state=123)

    # we will fit a model E[Y| D, X] by fitting a separate model for D==0
    # and a separate model for D==1. We do that for each model type in modely_list
    yhats0, yhats1 = np.zeros((y.shape[0], len(modely0_list))), np.zeros((y.shape[0], len(modely1_list)))
    for train, test in cv.split(X, y):
        for it, modely0 in enumerate(modely0_list):
            yhats0[test, it] = clone(modely0).fit(X.iloc[train][D[train]==0], y[train][D[train]==0]).predict(X.iloc[test])
        for it, modely1 in enumerate(modely1_list):
            yhats1[test, it] = clone(modely1).fit(X.iloc[train][D[train]==1], y[train][D[train]==1]).predict(X.iloc[test])

    # calculate stacking weights for the outcome model for each population
    # and combine the outcome model predictions
    yhat0 = clone(stacker).fit(yhats0[D==0], y[D==0]).predict(yhats0)
    yhat1 = clone(stacker).fit(yhats1[D==1], y[D==1]).predict(yhats1)

    # prediction for observed treatment using the stacked model
    yhat = yhat0 * (1 - D) + yhat1 * D

    # propensity scores
    Dhats = np.array([cross_val_predict(modeld, X, D, cv=cv, method='predict_proba', n_jobs=-1)[:, 1]
                     for modeld in modeld_list]).T
    # construct coefficients on each model based on stacker
    Dhat = clone(stacker).fit(Dhats, D).predict(Dhats)
    # trim propensities
    Dhat = np.clip(Dhat, trimming, 1 - trimming)

    # doubly robust quantity for every sample
    drhat = yhat1 - yhat0 + (y - yhat) * (D/Dhat - (1 - D)/(1 - Dhat))
    point = np.mean(drhat)
    var = np.var(drhat)
    stderr = np.sqrt(var / X.shape[0])
    return point, stderr, yhat, Dhat, y - yhat, D - Dhat, drhat

In [ ]:
result = dr_dirty(X, D, y, [lassoy, rfy, dtry, gbfy], [lassoy, rfy, dtry, gbfy], [lgrd, rfd, dtrd, gbfd], nfolds=5)

In [ ]:
tabledr = pd.concat([tabledr , summary(*result,  X, D, y, name='stacked (semi-cfit)')])
tabledr

We can compare the results between the PLR model and IRM model. We find that the effect under the IRM model is typically of lower value.

In [ ]:
table

In [ ]:
tabledr

# Using the EconML Library



We are interested in valid estimators of the average treatment effect of `e401` on `net_tfa`. There exist nice packages out there that can help us do our estimation with the simple call of a function. Such packages include `EconML` (Python), `DoubleML` (Python and R), and `ddml` (Stata and R).

We run through PLR and IRM using `EconML` below to illustrate.

In [ ]:
# code verified with this version of econml
# If you want to try out the latest econml version remove the version number;
# albeit mistakes could potentially be raised due to library updates”
!pip install econml==0.14.1 #

In [ ]:
# for these libraries we will just pre-featurize the controls
W = StandardScaler().fit_transform(transformer.fit_transform(X))

In [ ]:
from econml.dml import LinearDML
cv = KFold(n_splits=5, shuffle=True, random_state=123)
ldml = LinearDML(model_y=LassoCV(cv=cv), model_t=LogisticRegressionCV(cv=cv),
                 cv=3, discrete_treatment=True, random_state=123).fit(y, D, W=W)

In [ ]:
ldml.summary()

In [ ]:
# r2scores
r2scorey = np.mean(ldml.nuisance_scores_y)
r2scored = np.mean(ldml.nuisance_scores_t)
r2scorey, r2scored

In [ ]:
from econml.dr import LinearDRLearner

# dr learner in econml fits a single regression function of Y from X, D
# using all the data
dr = LinearDRLearner(model_regression=LassoCV(cv=cv),
                     model_propensity=LogisticRegressionCV(cv=cv),
                     cv=3, min_propensity=0.01, random_state=123).fit(y, D, W=W)

In [ ]:
dr.summary(T=1)

In [ ]:
from econml.dr import LinearDRLearner
from econml.utilities import SeparateModel

# to implement the separate regression models we need use the separate model wrapper
# that splits the data based on the last covariate (in this case D) and fits a separate
# model for each group. The input to the wrapper is the model to use for each value
# of the last covariate.
dr = LinearDRLearner(model_regression=SeparateModel(LassoCV(cv=cv), LassoCV(cv=cv)),
                     model_propensity=LogisticRegressionCV(cv=cv),
                     cv=3, min_propensity=0.01, random_state=123).fit(y, D, W=W)

In [ ]:
dr.summary(T=1)

# Using the DoubleML library

We now play with `DoubleML` to illustrate its use.

In [ ]:
!pip install doubleml

In [ ]:
from doubleml import DoubleMLData
dml_data = DoubleMLData.from_arrays(W, y, D)
print(dml_data)

In [ ]:
import doubleml as dml

dml_plr_obj = dml.DoubleMLPLR(dml_data, LassoCV(cv=cv), LogisticRegressionCV(cv=cv), n_folds=3)
print(dml_plr_obj.fit())

In [ ]:
dml_irm_obj = dml.DoubleMLIRM(dml_data, LassoCV(cv=cv), LogisticRegressionCV(cv=cv), n_folds=3)

print(dml_irm_obj.fit())